In [1]:
from    pulp    import *         


In [2]:
# Problem

model = LpProblem(  name    = "aggregate-planning", 
                    sense   = LpMinimize            )
                    

In [3]:
# Sets

T       = range(1,13)                   
T_ext   = range(13)                    
M       = ("b", "p")                    


In [4]:
# New set: index of the months during which extra workforce can be brought

EM = [7, 8]     # December and January


In [5]:
# Parameters

D = (   (28, 14), 
        (20, 10),
        (26, 4),
        (24, 4),
        (18, 6),
        (10, 8),
        (22, 10),
        (20, 12),
        (18, 6),
        (32, 8),
        (34, 14),
        (36, 6)     )           

wInit   = 86                    
h       = 800                   
f       = 1400                  

c       = (250, 500)            
iInit   = (8, 3)                

pCost   = (6250, 9750)          

pWf     = (380, 530)           
p       = 180                  

r       = 420                  
o       = (420/180)*1.5*40     

oLim    = 40
                    

In [6]:
# New parameter: cost of bringing temporary workers

e = 15 # [$ / hours]


In [7]:
# Decision variables

X = {   (i, t): LpVariable( name="X-{model}-{month}".format(    model   = M[i],                                     
                                                                month   = t     if t>=10 
                                                                                else "0{}".format(t)),              
                            lowBound=0                                                                 )
                                                                                for i in range(len(M)) for t in T       }


I = {   (i, t): LpVariable( name="I-{model}-{month}".format(    model   = M[i],                                     
                                                                month   =   t   if t>=10 
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for i in range(len(M)) for t in T_ext   }


W = {   (t): LpVariable(    name="W-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T_ext                          }


H = {   (t): LpVariable(    name="H-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }


F = {   (t): LpVariable(    name="F-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }


O = {   (t): LpVariable(    name="O-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in T                              }


In [8]:
# New decision variable: # of extra working hours supplied by temporary workers

E = {   (t): LpVariable(    name="E-{}"             .format(                t   if t>=10                            
                                                                                else "0{}".format(t)),
                            lowBound=0                                                               )              
                                                                                for t in EM                          }
                                                                                

In [9]:
# Modified objective function

model += lpSum(
                [   pCost[i]*X[(i,t)]   +                                       
                    c[i]*I[(i,t)]                                               
                                            for t in T                          
                                            for i in range(len(M)   )   ] +      
                [   r*W[t] + o*O[t]     +                                       
                    h*H[t] + f*F[t]                                                  
                                            for t in T                  ] +
                [   e*E[t]                                                
                                            for t in EM                 ]           # Extra workforce costs                           
                                                                            )


In [10]:
# Constraints


model += (W[0]      ==  wInit                                                                                                    
                                                                                )


for i in range(len(M)):                                                                         
    model += (I[(i, 0)] ==  iInit[i]
                                                                                )


for i in range(len(M)):                                                                         
    for t in T:
        model += (  I[(i,t-1)] + X[(i,t)] == I[(i,t)] + D[t-1][i]
                                                                                )


for t in T:                                                                                    
    model += (  W[t] == W[t-1] + H[t] - F[t]
                                                                                )


for t in T:                                                                                     
    model += (  O[t] <= W[t]
                                                                                )            


In [11]:
# Modified capacity constraint

for t in T:                                                                                     
    if t in EM:
        model += (  lpSum(X[(i,t)]*pWf[i] for i in range(len(M))) <= p*W[t] + oLim*O[t] + E[t]                             
                                                                                                )
    else:
        model += (  lpSum(X[(i,t)]*pWf[i] for i in range(len(M))) <= p*W[t] + oLim*O[t]     
                                                                                                )
                                                                                                

In [12]:
model

aggregate-planning:
MINIMIZE
15*E_07 + 15*E_08 + 1400*F_01 + 1400*F_02 + 1400*F_03 + 1400*F_04 + 1400*F_05 + 1400*F_06 + 1400*F_07 + 1400*F_08 + 1400*F_09 + 1400*F_10 + 1400*F_11 + 1400*F_12 + 800*H_01 + 800*H_02 + 800*H_03 + 800*H_04 + 800*H_05 + 800*H_06 + 800*H_07 + 800*H_08 + 800*H_09 + 800*H_10 + 800*H_11 + 800*H_12 + 250*I_b_01 + 250*I_b_02 + 250*I_b_03 + 250*I_b_04 + 250*I_b_05 + 250*I_b_06 + 250*I_b_07 + 250*I_b_08 + 250*I_b_09 + 250*I_b_10 + 250*I_b_11 + 250*I_b_12 + 500*I_p_01 + 500*I_p_02 + 500*I_p_03 + 500*I_p_04 + 500*I_p_05 + 500*I_p_06 + 500*I_p_07 + 500*I_p_08 + 500*I_p_09 + 500*I_p_10 + 500*I_p_11 + 500*I_p_12 + 140.0*O_01 + 140.0*O_02 + 140.0*O_03 + 140.0*O_04 + 140.0*O_05 + 140.0*O_06 + 140.0*O_07 + 140.0*O_08 + 140.0*O_09 + 140.0*O_10 + 140.0*O_11 + 140.0*O_12 + 420*W_01 + 420*W_02 + 420*W_03 + 420*W_04 + 420*W_05 + 420*W_06 + 420*W_07 + 420*W_08 + 420*W_09 + 420*W_10 + 420*W_11 + 420*W_12 + 6250*X_b_01 + 6250*X_b_02 + 6250*X_b_03 + 6250*X_b_04 + 6250*X_b_05 + 6250*

In [13]:
model.solve()
LpStatus[model.status]

'Optimal'

In [14]:
print("z* = ", value(model.objective))

z* =  3143976.3829999994


In [15]:
for v in model.variables():
    print(v.name, " = ", v.varValue)

E_07  =  0.0
E_08  =  0.0
F_01  =  11.3889
F_02  =  2.94444
F_03  =  0.0
F_04  =  0.0
F_05  =  0.0
F_06  =  0.0
F_07  =  0.0
F_08  =  0.0
F_09  =  0.0
F_10  =  0.0
F_11  =  0.0
F_12  =  0.0
H_01  =  0.0
H_02  =  0.0
H_03  =  0.0
H_04  =  0.0
H_05  =  0.0
H_06  =  0.0
H_07  =  0.0
H_08  =  0.0
H_09  =  4.05833
H_10  =  0.0
H_11  =  0.0
H_12  =  0.0
I_b_00  =  8.0
I_b_01  =  0.0
I_b_02  =  0.0
I_b_03  =  0.0
I_b_04  =  0.0
I_b_05  =  0.0
I_b_06  =  12.7895
I_b_07  =  10.7895
I_b_08  =  8.0
I_b_09  =  17.5013
I_b_10  =  10.2132
I_b_11  =  0.527632
I_b_12  =  0.0
I_p_00  =  3.0
I_p_01  =  0.0
I_p_02  =  0.0
I_p_03  =  0.0
I_p_04  =  0.0
I_p_05  =  0.0
I_p_06  =  0.0
I_p_07  =  0.0
I_p_08  =  0.0
I_p_09  =  0.0
I_p_10  =  0.0
I_p_11  =  0.0
I_p_12  =  0.0
O_01  =  0.0
O_02  =  0.0
O_03  =  0.0
O_04  =  0.0
O_05  =  0.0
O_06  =  0.0
O_07  =  0.0
O_08  =  0.0
O_09  =  0.0
O_10  =  0.0
O_11  =  75.725
O_12  =  75.725
W_00  =  86.0
W_01  =  74.6111
W_02  =  71.6667
W_03  =  71.6667
W_04  =  71.